In [1]:
import re
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pymongo
import json

# Connect to Mongo and Get Databases

mongo_connect_string = 'INSERT AMAZON STRING HERE'

client = pymongo.MongoClient(mongo_connect_string)
db = client.TwitterIPO
RawTweets = db.RawTweets
ProcessedTweets = db.ProcessedTweets
PriceData = db.PriceData
Vader = db.Vader
MNB_Logit_SVM_Sentiment = db.MNB_Logit_SVM_Sentiment
TweetWordCount = db.TweetWordCount
FinalDataset = db.FinalDataset

In [6]:
# Get Price Date and Create a Dataframe
prices = [x for x in PriceData.find()]
df_prices = pd.DataFrame(prices)
df_prices['Date'] = pd.to_datetime(df_prices.Date)
# Format the date
df_prices['Date'] = df_prices.Date.apply(lambda x: datetime.utcfromtimestamp(x.value / 1e9).date())
df_prices.head()

In [8]:
# Get Twitter Data
df_tweets = pd.DataFrame(list(ProcessedTweets.find({'lang' : 'en'})))
df_tweets.head()
print(df_tweets.shape)

In [1]:
# Get Sentiment Data

In [10]:
df_MNBLogSVM = pd.DataFrame(list(MNB_Logit_SVM_Sentiment.find()))
df_MNBLogSVM.shape

In [ ]:
# Merge MNBLogSVM Data with Twitter Data
df_senti = pd.merge(df_tweets, df_MNBLogSVM, on='UID',how='inner')

In [ ]:
df_senti.shape

In [ ]:
# Convert Merged Date to Date format for joining
df_senti.datepy = df_senti.datepy.map(lambda x: x.date())

In [ ]:
# Map into Positive and Negative
df_senti['logit_label'] = df_senti.logit_predict.map({1 : 'Positive', 0 : 'Negative'})
df_senti['mnb_label'] = df_senti.mnb_predict.map({1 : 'Positive', 0 : 'Negative'})
df_senti['svm_label'] = df_senti.svm_predict.map({1 : 'Positive', 0 : 'Negative'})

In [15]:
# Keep columns that we want. Get rid of the rest. Divide into Seperate Tables for SVM, LOGIT & MNB for easier joining.

In [16]:
df_svm = df_senti[['company', 'datepy', 'svm_label']]

In [17]:
df_mnb = df_senti[['company', 'datepy', 'mnb_label']]

In [18]:
df_logit = df_senti[['company', 'datepy', 'logit_label']]

In [ ]:
# Get Counts of each

In [19]:
svm_counts = df_svm.groupby(by=['company','datepy'])
svm_counts = svm_counts.svm_label.value_counts()

In [20]:
mnb_counts = df_mnb.groupby(by=['company','datepy'])
mnb_counts = mnb_counts.mnb_label.value_counts()

In [21]:
logit_counts = df_logit.groupby(by=['company','datepy'])
logit_counts = logit_counts.logit_label.value_counts()

In [ ]:
# Turn into dicts for joining

In [22]:
svm_dict = []
for k,v in svm_counts.iteritems():
    s = {}
    s['Name'] = k[0]
    s['Date'] = k[1]
    
    if k[2] == 'Positive':
        s['SVM_Pos_Count'] = v
    else:
        s['SVM_Neg_Count'] = v

    svm_dict.append(s)

In [23]:
mnb_dict = []
for k,v in mnb_counts.iteritems():
    s = {}
    s['Name'] = k[0]
    s['Date'] = k[1]
    
    if k[2] == 'Positive':
        s['MNB_Pos_Count'] = v
    else:
        s['MNB_Neg_Count'] = v

    mnb_dict.append(s)

In [24]:
logit_dict = []
for k,v in logit_counts.iteritems():
    
    log_res = {}
    
    log_res['Name'] = k[0]
    
    log_res['Date'] = k[1]
    
    if k[2] == 'Positive':
        log_res['Logit_Pos_Count'] = v
    else:
        log_res['Logit_Neg_Count'] = v

    logit_dict.append(log_res)

In [ ]:
# Convert back to dataframes and reset index

In [25]:
svm_counts = pd.DataFrame(svm_dict)
svm_counts = svm_counts.groupby(['Name','Date']).sum()
svm_counts = svm_counts.fillna(0)
svm_counts = svm_counts.reset_index(level=['Date','Name'])

In [26]:
mnb_counts = pd.DataFrame(mnb_dict)
mnb_counts = mnb_counts.groupby(['Name','Date']).sum()
mnb_counts = mnb_counts.fillna(0)
mnb_counts = mnb_counts.reset_index(level=['Date','Name'])

In [27]:
logit_counts = pd.DataFrame(logit_dict)
logit_counts = logit_counts.groupby(['Name','Date']).sum()
logit_counts = logit_counts.fillna(0)
logit_counts = logit_counts.reset_index(level=['Date','Name'])

<h2> TweetWordCount - Average Ratio & Counts </h2>

In [28]:
# Get Tweet Word Count and merge with Processed Tweets
df_tweetwordcount = pd.DataFrame(list(TweetWordCount.find()))
df_tweetwordcount = pd.merge(df_tweets, df_tweetwordcount, on='UID',how='inner')

# Convert Datetime object to date
df_tweetwordcount.datepy = df_tweetwordcount.datepy.map(lambda x: x.date())

# Add Labels
df_tweetwordcount['twc_label'] = np.where(df_tweetwordcount['ratio'] > 0, 'Positive','Negative')
df_tweetwordcount.loc[df_tweetwordcount.ratio == 0, 'twc_label'] = 'Neutral'

# Groupby Date and Company for Average Ratio
df_tweetwordcount_ratio = df_tweetwordcount.groupby(['datepy','company']).mean()

# Keep Date, Company & Ratio
df_tweetwordcount_ratio = df_tweetwordcount_ratio[['ratio']]
df_tweetwordcount_ratio = df_tweetwordcount_ratio.reset_index(['datepy','company'])
df_tweetwordcount_ratio = df_tweetwordcount_ratio.rename(columns={'datepy' : 'Date','company':'Name','ratio' : 'WordCountRatio'})

In [29]:
# Get Counts

df_tweetwordcount_counts = df_tweetwordcount[['company', 'datepy', 'twc_label']]

twc_counts = df_tweetwordcount_counts.groupby(by=['company','datepy'])
twc_counts = twc_counts.twc_label.value_counts()

In [30]:
twc_dict = []
for k,v in twc_counts.iteritems():
    s = {}
    s['Name'] = k[0]
    s['Date'] = k[1]
    
    if k[2] == 'Positive':
        s['TWC_Pos_Count'] = v
    elif k[2] == 'Neutral':
        s['TWC_Neutral_Count'] = v
    else:
        s['TWC_Neg_Count'] = v

    twc_dict.append(s)

In [31]:
twc_counts = pd.DataFrame(twc_dict)
twc_counts = twc_counts.groupby(['Name','Date']).sum()
twc_counts = twc_counts.fillna(0)
twc_counts = twc_counts.reset_index(level=['Date','Name'])

<h2> Vader Incorporation </h2>

In [32]:
# Get Tweet Word Count and merge with Processed Tweets
df_vader = pd.DataFrame(list(Vader.find()))
df_vader = pd.merge(df_tweets, df_vader, on='UID',how='inner')

# Convert Datetime object to date
df_vader.datepy = df_vader.datepy.map(lambda x: x.date())

# Add Labels
df_vader['vader_label'] = np.where(df_vader['compound'] > 0, 'Positive','Negative')
criterion = df_vader['compound'].map(lambda x: x == 0)
df_vader['vader_label'][criterion] = 'Neutral'

# Groupby Date and Company for Average compound
df_vader_score = df_vader.groupby(['datepy','company']).mean()

# Keep Date, Company & compound
df_vader_score = df_vader_score[['compound','neg','neu','pos']]
df_vader_score = df_vader_score.reset_index(['datepy','company'])
df_vader_score = df_vader_score.rename(columns={'datepy' : 'Date',
                                                'company':'Name',
                                                'compound' : 'Vader_Compound',
                                                'neg' : 'Vader_Neg',
                                                'neu': 'Vader_Neu',
                                                'pos' : 'Vader_Pos'})

C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
## Get Vader Label Counts

df_vader_counts = df_vader[['company', 'datepy', 'vader_label']]

vader_counts = df_vader_counts.groupby(by=['company','datepy'])
vader_counts = vader_counts.vader_label.value_counts()

In [34]:
vader_dict = []
for k,v in vader_counts.iteritems():
    
    s = {}
    s['Name'] = k[0]
    s['Date'] = k[1]
    
    if k[2] == 'Positive':
        s['Vader_Pos_Count'] = v
    elif k[2] == 'Neutral':
        s['Vader_Neutral_Count'] = v
    else:
        s['Vader_Neg_Count'] = v

    vader_dict.append(s)

In [35]:
vader_counts = pd.DataFrame(vader_dict)
vader_counts = vader_counts.groupby(['Name','Date']).sum()
vader_counts = vader_counts.fillna(0)
vader_counts = vader_counts.reset_index(level=['Date','Name'])

<h2> Merge all Sentiment Datasets</h2>

In [36]:
## Merge Counts

In [37]:
df_senti_combined = pd.merge(svm_counts, mnb_counts, on=['Date','Name'], how='inner')

In [38]:
df_senti_combined = pd.merge(df_senti_combined,logit_counts, on=['Date','Name'], how='inner')

In [39]:
df_senti_combined = pd.merge(df_senti_combined,twc_counts, on=['Date','Name'], how='inner')

In [40]:
df_senti_combined = pd.merge(df_senti_combined,vader_counts, on=['Date','Name'], how='inner')

In [41]:
## Merge Scores, Ratios

In [42]:
df_senti_combined = pd.merge(df_senti_combined,df_tweetwordcount_ratio, on=['Date','Name'], how='inner')

In [43]:
df_senti_combined = pd.merge(df_senti_combined,df_vader_score, on=['Date','Name'], how='inner')

In [44]:
## Shift date by +1 to merge with price data of next day

In [45]:
df_senti_combined['Date'] = df_senti_combined.Date.map(lambda x: x + timedelta(days=1))

<h2> Combine Sentiment and Price </h2>

In [46]:
df_combined = pd.merge(df_prices,df_senti_combined, on=['Date','Name'], how='outer')

In [47]:
df_combined = df_combined.dropna(how='any')

In [48]:
df_combined.shape

(807, 36)

In [810]:
## Write to CSV for testing in WEKA?
# df_combined.to_csv('test_senti.csv')

In [68]:
## Insert into MongoDB (Needs Date converstion to Datetime)
df_combined.Date = df_combined.Date.map(lambda t: datetime(t.year, t.month, t.day))
FinalDataset.insert_many(df_combined.to_dict('records'))